# [모듈 5.1] 평가 스텝 개발

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 
%store

Stored variables and their in-db values:
base_preproc_input_dir                 -> 'opt/ml/processing/input'
dataset_path                           -> 'opt/ml/processing/input/dataset.csv'
default_bucket                         -> 'sagemaker-ap-northeast-2-057716757052'
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
preprocessing_code_dir                 -> 'fraud/preprocessing.py'
processing_instance_count              -> ParameterInteger(name='ProcessingInstanceCount', p
project_prefix                         -> 'fraud2scratch'
s3_dataset_path                        -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
test_preproc__dir_artifact             -> 's3://sagemaker-ap-northeast-2-057716757052/sklear
train_model_artifact                   -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
train_preproc_dir_artifact             -> 's3://sagemaker-ap-northeast-2-057716757052/sklear
val_preproc_dir_artifact               -> 's3:

## 1. 로컬에서 스크립트 실행

### 환경 셋업

In [2]:
import os
base_dir = 'opt/ml/processing'
os.makedirs(base_dir, exist_ok=True)

output_evaluation_dir = 'opt/ml/processing/evaluation'
os.makedirs(output_evaluation_dir, exist_ok=True)

# 훈련 아티펙트를 다운로드 하여 로컬 저장
base_model_dir = 'opt/ml/processing/model'
base_model_path = f"{base_model_dir}/model.tar.gz"
os.makedirs(base_model_dir, exist_ok=True)

! aws s3 cp  {train_model_artifact} {base_model_dir}

# 테스트 데이터 세트의 파일 경로 기술
base_test_path = f"{test_preproc__dir_artifact}/test.csv"
print("base_test_path: ", base_test_path)


download: s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/model/pipelines-tfr3929pycdw-FraudScratchTrain-YBzZARNFEU/output/model.tar.gz to opt/ml/processing/model/model.tar.gz
base_test_path:  s3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-04-13-03-08-45-278/output/test/test.csv


In [3]:
print("base_dir: \n", base_dir)
print("base_model_path: \n", base_model_path)
print("base_test_path: \n", base_test_path)
print("output_evaluation_dir: \n", output_evaluation_dir)

base_dir: 
 opt/ml/processing
base_model_path: 
 opt/ml/processing/model/model.tar.gz
base_test_path: 
 s3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-04-13-03-08-45-278/output/test/test.csv
output_evaluation_dir: 
 opt/ml/processing/evaluation


### 로컬에서 스크립트 실행

In [4]:
%%sh -s "$base_dir" "$base_model_path" "$base_test_path" "$output_evaluation_dir"
python fraud/evaluation.py \
--base_dir $1 \
--model_path $2 \
--test_path $3 \
--output_evaluation_dir $4


#############################################
args.model_path: opt/ml/processing/model/model.tar.gz
args.test_path: s3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-04-13-03-08-45-278/output/test/test.csv
args.output_evaluation_dir: opt/ml/processing/evaluation
****** All folder and files under opt/ml/processing ****** 
('opt/ml/processing', ['validation', 'train', 'test', 'evaluation', 'input', 'output', 'model'], [])
('opt/ml/processing/validation', [], ['validation.csv'])
('opt/ml/processing/train', [], ['train.csv'])
('opt/ml/processing/test', [], ['test.csv'])
('opt/ml/processing/evaluation', [], ['evaluation.json'])
('opt/ml/processing/input', ['.ipynb_checkpoints'], ['dataset.csv'])
('opt/ml/processing/input/.ipynb_checkpoints', [], [])
('opt/ml/processing/output', ['validation', 'train', 'test'], [])
('opt/ml/processing/output/validation', [], ['validation.csv'])
('opt/ml/processing/output/train', [], ['train.csv'])
('opt/ml/processing/output/test', [], ['t

## 2. 로컬 다커 에서 모델 평가
xgboost 내장 알고리즘 컨테이너는 local mode (로컬에서 다커 실행)을 지원하지 않기에 Scikit-learn를 기본 이미지로 가져오고, 스크립트에서 xgboost를 설치하여 사용함

In [5]:
# scikit learn 다커 이미지
image_uri = '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

In [6]:
from sagemaker.processing import ScriptProcessor

processing_instance_type = 'local'
eval_script_processor = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-fraud-scratch-eval",
    role=role,
)

In [7]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

eval_script_processor.run(
                        inputs=[
                            ProcessingInput(
#                                source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
                                source= train_model_artifact,  # model_artifcat_path,
                                destination="/opt/ml/processing/model"
                            ),
                            ProcessingInput(
#                                 source=step_process.properties.ProcessingOutputConfig.Outputs[
#                                     "test"
#                                 ].S3Output.S3Uri,
                                source = test_preproc__dir_artifact, # prep_test_output,
                                destination="/opt/ml/processing/test"
                            )
                        ],
                        outputs=[
                            ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
                        ],
                        code="fraud/evaluation.py",
)


Job Name:  script-fraud-scratch-eval-2021-04-13-07-23-30-415
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/model/pipelines-tfr3929pycdw-FraudScratchTrain-YBzZARNFEU/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-04-13-03-08-45-278/output/test', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/script-fraud-scratch-eval-2021-04-13-07-23-30-415/input/code/evaluation.py', 'LocalPath': '/opt/ml/processing/inp

## 3. 모델 빌딩 파이프라인에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [8]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

### 학습모델을 평가하기 위한 모델 평가단계 정의 

먼저 모델 평가용 프로세싱 단계에서 질행할 평가용 스크립트를 작성합니다. 

파이프라인 실행 후 `evaluation.json`파일을 통해 평가결과를 확인할 수 있습니다.

평가 스크립트는 `xgboost`를 사용하고 다음을 실행합니다.

* 모델을 로드합니다. 
* 테스트 데이터를 읽습니다. 
* 테스트 데이터에 대한 예측을 실행합니다. 
* mse 등을 포함하는 분류보고서를 작성합니다. 

In [9]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-fraud2scratch-eval",
    role=role,
)

In [10]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep


# evaluation_report = PropertyFile(
#     name="EvaluationReport",
#     output_name="evaluation",
#     path="evaluation.json"
# )
step_eval = ProcessingStep(
    name="FraudEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source= train_model_artifact,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source= test_preproc__dir_artifact,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="fraud/evaluation.py",
#    property_files=[evaluation_report],
)

### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [11]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"FraudScratch"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
    ],
    steps=[step_eval],
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [12]:
import json


definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [13]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraudscratch',
 'ResponseMetadata': {'RequestId': 'e6414334-68d6-4bf1-a0ea-308fbec24ec3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e6414334-68d6-4bf1-a0ea-308fbec24ec3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'date': 'Tue, 13 Apr 2021 07:23:45 GMT'},
  'RetryAttempts': 0}}

In [14]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [15]:
execution.describe()
# execution.wait()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraudscratch',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraudscratch/execution/d7tbbeb35rbw',
 'PipelineExecutionDisplayName': 'execution-1618298626788',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 4, 13, 7, 23, 46, 706000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 4, 13, 7, 23, 46, 706000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '85bafe78-5562-421a-a7e8-ed82169b9dcd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '85bafe78-5562-421a-a7e8-ed82169b9dcd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '399',
   'date': 'Tue, 13 Apr 2021 07:23:45 GMT'},
  'RetryAttempts': 0}}

In [16]:
execution.list_steps()

[]